In [ ]:
!pip install tensorflow
!pip install tensorflow-gpu
!pip install nltk

     |████████████████████████████████| 394.3MB 39kB/s 


In [1]:
import random
from keras.optimizers import SGD
from keras.layers import Dense, Dropout
from keras.models import load_model
from keras.models import Sequential
import numpy as np
import pickle
import json
import nltk
from nltk.stem import WordNetLemmatizer

In [2]:
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [4]:
words = []
classes = []
documents = []
ignore_words = ["?", "!"]
intents = {
    'intents': [{
      'tag': 'greeting',
      'patterns': ["hi there", "hello","haroo","yaw","wassup", "hi", "hey", "holla", "hello","namaste","namastey","are you there"],
      'responses': ["hello, hope you are fine!","Hi, how can I help you?","Hi, I would love to help you."],
      'context': [""]  
    },
    {
        'tag': "goodbye",
        'patterns': ["bye", "good bye", "see you later"],
        'responses': ["have a nice time.", "bye bye."],
        'context': [""]
    },
    {
        'tag': "thanks",
        'patterns': ["Thanks", "okay","Thank you","thankyou", "That's helpful", "Awesome, thanks", "Thanks for helping me", "wow", "great"],
        'responses': ["Happy to help!", "Any time!","you're welcome", "My pleasure"],
        'context': [""]        
    },
    {
        'tag': "no_answer",
        'patterns': [""],
        'responses': ["Sorry, I didn't understand you.", "Please give me more info.", "Not sure I understand that."],
        'context': [""]
    },
    {
        'tag': "ask_name",
        'patterns': ["what's your name?","who are you?","can you tell something about you?","help"],
        'responses': ["I'm Covid Bot. You can ask me anything related to Covid-19. I will try my best to help you. You can also say remind me "],
        'context': [""]
    },
    {
        'tag': "name",
        'patterns': ["my name is ", "I'm ","I am"],
        'responses': ["Oooh great to meet you {n}. Please let me know if I can help you.","Great! How may I assist you {n}", "Oh, I'll keep that in mind {n}."],
        'context': [""]
    },
    {
        'tag': "covid_info",
        'patterns': ["What is a coronavirus?", "What is coronavirus? ","What is covid-19?", "what is covid", "what is covid 19"],
        'responses': ["Coronaviruses are a large family of viruses which may cause illness in animals or humans. In humans, several coronaviruses are known to cause respiratory infections ranging from the common cold to more severe diseases", "COVID-19 is the infectious disease caused by the most recently discovered coronavirus. This new virus and disease were unknown before the outbreak began in Wuhan, China, in December 2019."],
        'context': [""]
    },
    {
        'tag': "covid_symptoms",
        'patterns' : ["What are the symptoms of COVID-19?", "What are the symptoms of COVID?", "What are the symptoms of coronavirus?", "symptoms of", "symptoms?"],
        'responses' : ["The most common symptoms of COVID-19 are fever, tiredness, and dry cough. Some patients may have aches and pains, nasal congestion, runny nose, sore throat or diarrhea. These symptoms are usually mild and begin gradually. Some people become infected but don’t develop any symptoms and don't feel unwell. Most people (about 80%) recover from the disease without needing special treatment. Around 1 out of every 6 people who gets COVID-19 becomes seriously ill and develops difficulty breathing. Older people, and those with underlying medical problems like high blood pressure, heart problems or diabetes, are more likely to develop serious illness. People with fever, cough and difficulty breathing should seek medical attention."],
        'context' : [""]
    },
    {
        'tag': "spread",
        'patterns': ["How does COVID-19 spread?", "How does COVID spread?", "How does coronavirus spread?", "How does corona spread?", "corona spread", "spreading of corona?", "spreading of covid"],
        'responses': ["People can catch COVID-19 from others who have the virus. The disease can spread from person to person through small droplets from the nose or mouth which are spread when a person with COVID-19 coughs or exhales. People can also catch COVID-19 if they breathe in droplets from a person with COVID-19 who coughs out or exhales droplets. This is why it is important to stay more than 1 meter (3 feet) away from a person who is sick."],
        'context': [""]
    },
    {
        'tag': "transmission_medium",
        'patterns': ["can the virus transmit through air?", "transmission of covid", "transmission", "Can the virus that causes COVID-19 be transmitted through the air?"],
        'responses': ["Studies to date suggest that the virus that causes COVID-19 is mainly transmitted through contact with respiratory droplets rather than through the air. "],
        'context': [""]
    },
    {
        'tag': "no_symptoms_spread",
        'patterns': ["Can CoVID-19 be caught from a person who has no symptoms?", "Can CoVID be caught from a person who has no symptoms?", "Can CoVID19 be caught from a person who has no symptoms?", "Can Corona be caught from a person who has no symptoms?", "Can coronavirus be caught from a person who has no symptoms?", "Can I catch covid from a person who has no symptoms?", "Can I catch covid-19 from a person who has no symptoms?", "Can I catch coronavirus from a person who has no symptoms?","no symptoms"],
        'responses': ["The main way the disease spreads is through respiratory droplets expelled by someone who is coughing. The risk of catching COVID-19 from someone with no symptoms at all is very low.", "Many people with COVID-19 experience only mild symptoms. This is particularly true at the early stages of the disease. It is therefore possible to catch COVID-19 from someone who has, for example, just a mild cough and does not feel ill. "],
        'context': [""]
    },
    {
        'tag': "faeces_spread",
        'patterns': ["Can I catch COVID-19 from the faeces of someone", "Can I catch COVID from the faeces of someone", "Can I catch corona from the faeces of someone", "faeces"],
        'responses': ["The risk of catching COVID-19 from the feces of an infected person appears to be low. While initial investigations suggest the virus may be present in feces in some cases, spread through this route is not a main feature of the outbreak. "],
        'context': [""]
    },
    {
        'tag': "protect_and_prevent",
        'patterns': ["protect and prevent", "how to protect and prevent", "What can I do to protect myself and prevent the spread of disease?", "What can I do to protect and prevent the spread of covid", "What can I do to protect and prevent the spread of covid-19", "What can I do to protect and prevent the spread of corona", "What can I do to protect and prevent the spread of coronavirus"],
        'responses': ["1. Regularly and thoroughly clean your hands with an alcohol-based hand rub or wash them with soap and water.\n2. Maintain at least 1 metre (3 feet) distance between yourself and anyone who is coughing or sneezing.\n3. Avoid touching eyes, nose and mouth.\n4. Make sure you, and the people around you, follow good respiratory hygiene. This means covering your mouth and nose with your bent elbow or tissue when you cough or sneeze. Then dispose of the used tissue immediately.\n5. Stay home if you feel unwell. If you have a fever, cough and difficulty breathing, seek medical attention and call in advance. Follow the directions of your local health authority.\n6. If possible, avoid traveling to places  – especially if you are an older person or have diabetes, heart or lung disease.\n7. Keep up to date on the latest COVID-19 hotspots (cities or local areas where COVID-19 is spreading widely).\n8. Self-isolate by staying at home if you begin to feel unwell, even with mild symptoms such as headache, low grade fever (37.3 C or above) and slight runny nose, until you recover."],
        'context': [""]
    },
    {
        'tag': "protect_and_prevent_highalert",
        'patterns': ["What can I do to protect myself if I have recently visited areas where COVID-19 is spreading?" ,"Protection measures for persons who are in or have recently visited (past 14 days) areas where COVID-19 is spreading?", "Protection measures for persons who are in or have recently visited (past 14 days) areas where COVID-19 is spreading"],
        'responses': ["If you develop fever, cough and difficulty breathing, seek medical advice promptly as this may be due to a respiratory infection or other serious condition. Call in advance and tell your provider of any recent travel or contact with travelers.\nSelf-isolate by staying at home if you begin to feel unwell, even with mild symptoms such as headache, low grade fever (37.3 C or above) and slight runny nose, until you recover."],
        'context': [""]
    },
    {
        'tag': "likelihood_of_infection",
        'patterns': ["How likely am I to catch COVID-19?", "How likely am I to catch COVID19?", "How likely am I to catch COVID?", "How likely am I to catch coronavirus?", "How likely am I to catch corona?"],
        'responses': ["The risk depends on where you  are - and more specifically, whether there is a COVID-19 outbreak unfolding there.", "For most people in most locations the risk of catching COVID-19 is still low. However, there are now places around the world (cities or areas) where the disease is spreading. For people living in, or visiting, these areas the risk of catching COVID-19 is higher.\nCOVID-19 outbreaks can be contained and transmission stopped, as has been shown in China and some other countries. Unfortunately, new outbreaks can emerge rapidly. It’s important to be aware of the situation where you are or intend to go."],
        'context': [""]
    },
    {
        'tag': "worry",
        'patterns': ["Should I worry about coronavirus", "Should I worry about COVID19", "Should I worry about COVID-19", "Should I worry about corona", "worried about coronavirus", "Should I be worried about coronavirus", "Should I be worried about COVID19", "Should I be worried about COVID-19", "Should I be worried about corona", "Who is at risk of developing severe illness?"],
        'responses': ["Illness due to COVID-19 infection is generally mild, especially for children and young adults. However, it can cause serious illness: about 1 in every 5 people who catch it need hospital care.\nWe can channel our concerns into actions to protect ourselves, our loved ones and our communities. While we are still learning about how COVID-2019 affects people, older persons and persons with pre-existing medical conditions (such as high blood pressure, heart disease, lung disease, cancer or diabetes)  appear to develop serious illness more often than others. "],
        'context': [""]
    },
    {
        'tag': "antibodies",
        'patterns': ["are antibodies effective", "Are antibiotics effective in preventing or treating the covid?", "Are antibiotics effective in preventing or treating the coronavirus?", "Are antibiotics effective in preventing or treating the covid19?", "effectiveness of antibodies", "antibodies useful"],
        'responses': ["No. Antibiotics do not work against viruses, they only work on bacterial infections. COVID-19 is caused by a virus, so antibiotics do not work. ", "Antibiotics should not be used as a means of prevention or treatment of COVID-19. They should only be used as directed by a physician to treat a bacterial infection. "],
        'context': [""]
    },
    {
        'tag': "treatment",
        'patterns': ["Are there any medicines or therapies that can prevent or cure COVID-19?", "Are there any medicines or therapies that can prevent or cure COVID-19?", "Is there a vaccine, drug or treatment for COVID-19?", "Is there a vaccine, drug or treatment for COVID-19?", "Is there a vaccine, drug or treatment for COVID-19?", "Is there a vaccine, drug or treatment for COVID-19?"],
        'responses': ["Yes, a number of vaccines are currently being used for treating covid-19. Please register yourself in this site and take it from your nearest vaccination center"],
        'context': [""]
    },
    {
        'tag': "sars",
        'patterns': ["Is COVID-19 the same as SARS?", "Is COVID the same as SARS?", "Is coronavirus the same as SARS?", "similarility covid and sars", "similarity covid-19 and sars", "similarity coronavirus and sars"],
        'responses': ["SARS was more deadly but much less infectious than COVID-19. There have been no outbreaks of SARS anywhere in the world since 2003.", "No, the virus that causes COVID-19 and the one that caused the outbreak of Severe Acute Respiratory Syndrome (SARS) in 2003 are related to each other genetically, but the diseases they cause are quite different."],
        'context': [""]
    },
    {
        'tag': "mask",
        'patterns': ["Should I wear a mask to protect myself", "wear a mask", "wearing a mask", "need of mask", "is mask necessary", "is it safe to go outside without mask"],
        'responses': ["Only wear a mask if you are ill with COVID-19 symptoms (especially coughing) or looking after someone who may have COVID-19. Disposable face mask can only be used once.", "If you are not ill or looking after someone who is ill then you are wasting a mask. ", "There is a world-wide shortage of masks, so WHO urges people to use masks wisely."],
        'context': [""]
    },
    {
      'tag': 'Mask Removal',
      'patterns': ["How to put on, use, take off and dispose of a mask","How to remove mask","How to dispose mask","Remove mask","Dispose mask"],
      'responses': ["Remember, a mask should only be used by health workers, care takers, and individuals with respiratory symptoms, such as fever and cough.",
                    "Before touching the mask, clean hands with an alcohol-based hand rub or soap and water"],
      'context': [""]  
    },
    {
      'tag': 'Incubation period',
      'patterns': ["How long is the incubation period for COVID-19","Corona incubation period","COVID-19 incubation period"],
      'responses': ["Most estimates of the incubation period for COVID-19 range from 1-14 days, most commonly around five days. These estimates will be updated as more data become available.",
                    "The \"incubation period\" means the time between catching the virus and beginning to have symptoms of the disease."],
      'context': [""]  
    },
    {
      'tag': 'Animals',
      'patterns': ["Can humans be infected with the COVID-19 from an animal source","COVID-19 from animals","Covid from bats","Corona from animals","Corona from bats"],
      'responses': ["Coronaviruses are a large family of viruses that are common in animals. Occasionally, people get infected with these viruses which may then spread to other people.",
                    "SARS-CoV was associated with civet cats and MERS-CoV is transmitted by dromedary camels",
                    "Possible animal sources of COVID-19 have not yet been confirmed.\nTo protect yourself, such as when visiting live animal markets, avoid direct contact with animals and surfaces in contact with animals\nHandle raw meat, milk or animal organs with care to avoid contamination of uncooked foods and avoid consuming raw or undercooked animal products"],
      'context': [""]  
    },
    {
      'tag': 'Pet',
      'patterns': ["Can I catch COVID-19 from my pet?","Corona from pet","Corona from dog","Corona from cat"],
      'responses': ["While there has been one instance of a dog being infected in Hong Kong, to date, there is no evidence that a dog, cat or any pet can transmit COVID-19.",
                    "COVID-19 is mainly spread through droplets produced when an infected person coughs, sneezes, or speaks."],
      'context': [""]  
    },
    {
      'tag': 'Surface',
      'patterns': ["How long does the virus survive on surfaces","Corona on surface time","Time that corona stays on a surface"],
      'responses': ["It is not certain how long the virus that causes COVID-19 survives on surfaces, but it seems to behave like other coronaviruses. Studies suggest that coronaviruses (including preliminary information on the COVID-19 virus) may persist on surfaces for a few hours or up to several days. This may vary under different conditions (e.g. type of surface, temperature or humidity of the environment)\nIf you think a surface may be infected, clean it with simple disinfectant to kill the virus and protect yourself and others."],
      'context': [""]  
    },
    {
      'tag': 'Deliveries',
      'patterns': ["Is it safe to receive a package from any area where COVID-19 has been reported","Accepting delivery during corona","Is it safe to take deliveries during corona"],
      'responses': ["Yes, it is safe. The likelihood of an infected person contaminating commercial goods is low and the risk of catching the virus that causes COVID-19 from a package that has been moved, travelled, and exposed to different conditions and temperature is also low. "],
      'context': [""]  
    },
    {
      'tag': 'Dont',
      'patterns': ["Is there anything I should not do","Dont's in COVID-19","Dont's in corona"],
      'responses': ["1. Smoking\n2. Wearing multiple masks\n3. Taking antibiotics"],
      'context': [""]  
    },
    {
      'tag': 'Origin',
      'patterns': ["Is the source causing COVID-19 known","How did the first human SARS-CoV-2 infections occur","Origin of COVID-19","Origin of corona","Start of corona"],
      'responses': ["Currently, the source of SARS-CoV-2, the coronavirus (CoV) causing COVID-19 is unknown.\nThe first human cases of COVID-19 were identified in Wuhan City, China in December 2019.",
                    "SARS-CoV, the virus which caused the SARS outbreak in 2003, jumped from an animal reservoir (civet cats, a farmed wild animal) to humans and then spread between humans.\nUntil the source of this virus is identified and controlled, there is a risk of reintroduction of the virus in the human population and the risk of new outbreaks like the ones we are currently experiencing."],
      'context': [""]  
    },
    {
      'tag': 'Air',
      'patterns': ["Is COVID-19 airborne","Does corona spread from the air","Does COVID spread from the air","Corona in air"],
      'responses': ["The virus that causes COVID-19 is mainly transmitted through droplets generated when an infected person coughs, sneezes, or speaks. These droplets are too heavy to hang in the air. They quickly fall on floors or surfaces. "],
      'context': [""]  
    },
    {
      'tag': 'Pregnancy',
      'patterns': ["Are pregnant women at higher risk from COVID-19","Corona during pregnancy","COVID during pregnancy","Pregnant"],
      'responses': ["At present there is no evidence that they are at higher risk of severe illness than the general population.\nDue to changes in their bodies and immune systems, we know that pregnant women can be badly affected by some respiratory infections. It is therefore important that they take precautions to protect themselves against COVID-19, and report possible symptoms (including fever, cough or difficulty breathing) to their healthcare provider.",
                    "Pregnant women should take the same precautions to avoid COVID-19 infection as other people",
                    "Pregnant women and women who have recently delivered – including those affected by COVID-19 - should attend their routine care appointments"],
      'context': [""]  
    },
    {
      'tag': 'Pregnancy Testing',
      'patterns': ["Should pregnant women be tested for COVID-19"],
      'responses': ["Testing protocols and eligibility vary depending on where you live."],
      'context': [""]  
    },
    {
      'tag': 'Unborn',
      'patterns': ["Can COVID-19 be passed from a woman to her unborn baby"],
      'responses': ["We still do not know if a pregnant woman with COVID-19 can pass the virus to her foetus or baby during pregnancy or delivery. To date, the virus has not been found in samples of amniotic fluid or breastmilk."],
      'context': [""]  
    },
    {
      'tag': 'Pregnancy birth',
      'patterns': ["What care should be available during pregnancy and childbirth"],
      'responses': ["All pregnant women, including those with confirmed or suspected COVID-19 infections, have the right to high quality care before, during and after childbirth"],
      'context': [""]  
    },
    {
      'tag': 'ceasarean',
      'patterns': ["Do pregnant women with suspected or confirmed COVID-19 need to give birth by ceasarean section"],
      'responses': ["No. WHO advice is that caesarean sections should only be performed when medically justified."],
      'context': [""]  
    },
    {
      'tag': 'breastfeeding',
      'patterns': ["Can women with COVID-19 breastfeed"],
      'responses': ["Yes. Women with COVID-19 can breastfeed if they wish to do so."],
      'context': [""]  
    },
    {
      'tag': 'baby',
      'patterns': ["Can I touch and hold my newborn baby if I have COVID-19"],
      'responses': ["Yes. Close contact and early, exclusive breastfeeding helps a baby to thrive. "],
      'context': [""]  
    },
    {
      'tag': 'influenza',
      'patterns': ["How are COVID-19 and influenza viruses similar","How are corona and influenza viruses similar","Influenza and corona","How are COVID-19 and influenza viruses different"],
      'responses': ["COVID-19 and influenza viruses have a similar disease presentation. That is, they both cause respiratory disease, which presents as a wide range of illness from asymptomatic or mild through to severe disease and death.",
                    "Both viruses are transmitted by contact, droplets and fomites.\nHowever, the speed of transmission is an important point of difference between the two viruses. Influenza has a shorter median incubation period (the time from infection to appearance of symptoms) and a shorter serial interval (the time between successive cases) than COVID-19 virus.",
                    "Mortality for COVID-19 appears higher than for influenza, especially seasonal influenza.",
                    "Those most at risk for severe influenza infection are children, pregnant women, elderly, those with underlying chronic medical conditions and those who are immunosuppressed."],
      'context': [""]  
    },
    {
      'tag': 'medical',
      'patterns': ["What medical interventions are available for COVID-19 and influenza viruses"],
      'responses': ["While there are a number of therapeutics currently in clinical trials in China and more than 20 vaccines in development for COVID-19, there are currently no licensed vaccines or therapeutics for COVID-19."],
      'context': [""]  
    },
    {
      'tag': 'smoke_risk',
      'patterns': ["Are smokers and tobacco users at higher risk of COVID-19 infection","Cigarette in corona","Smoking in corona","Smoking in covid"],
      'responses': ["Smokers are likely to be more vulnerable to COVID-19 as the act of smoking means that fingers (and possibly contaminated cigarettes) are in contact with lips which increases the possibility of transmission of virus from hand to mouth.",
                    "Smoking products such as water pipes often involve the sharing of mouth pieces and hoses, which could facilitate the transmission of COVID-19 in communal and social settings."],
      'context': [""]  
    },
    {
      'tag': 'gathering',
      'patterns': ["How large does a meeting or event need to be in order to be a mass gathering","People gathering during corona"],
      'responses': ["High profile international sporting events such as the Olympics or World Cups as well as international religious events such as the Hajj count as mass gatherings.",
                    "An event counts as a “mass gatherings” if the number of people it brings together is so large that it has the potential to strain the planning and response resources of the health system in the community where it takes place. "],
      'context': [""]  
    }
    ]
}

In [5]:
for intent in intents["intents"]:
    for pattern in intent["patterns"]:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent["tag"]))
  
        # adding classes to our class list
        if intent["tag"] not in classes:
            classes.append(intent["tag"])

In [6]:
lemmatizer = WordNetLemmatizer()
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print(len(documents), "documents")

print(len(classes), "classes", classes)

print(len(words), "unique lemmatized words", words)


pickle.dump(words, open("words.pkl", "wb"))
pickle.dump(classes, open("classes.pkl", "wb"))

171 documents
40 classes ['Air', 'Animals', 'Deliveries', 'Dont', 'Incubation period', 'Mask Removal', 'Origin', 'Pet', 'Pregnancy', 'Pregnancy Testing', 'Pregnancy birth', 'Surface', 'Unborn', 'antibodies', 'ask_name', 'baby', 'breastfeeding', 'ceasarean', 'covid_info', 'covid_symptoms', 'faeces_spread', 'gathering', 'goodbye', 'greeting', 'influenza', 'likelihood_of_infection', 'mask', 'medical', 'name', 'no_answer', 'no_symptoms_spread', 'protect_and_prevent', 'protect_and_prevent_highalert', 'sars', 'smoke_risk', 'spread', 'thanks', 'transmission_medium', 'treatment', 'worry']
217 unique lemmatized words ["'m", "'s", '(', ')', ',', '14', '19', 'a', 'about', 'accepting', 'air', 'airborne', 'am', 'an', 'and', 'animal', 'antibiotic', 'antibody', 'any', 'anything', 'are', 'area', 'at', 'available', 'awesome', 'baby', 'bat', 'be', 'been', 'birth', 'breastfeed', 'by', 'bye', 'can', 'care', 'cat', 'catch', 'caught', 'cause', 'causing', 'ceasarean', 'childbirth', 'cigarette', 'confirmed', 

In [7]:
# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:, 0])
train_y = list(training[:, 1])
print("Training data created")

Training data created


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [8]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation="softmax"))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               27904     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 40)                2600      
Total params: 38,760
Trainable params: 38,760
Non-trainable params: 0
_________________________________________________________________


In [9]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])

In [10]:
from keras import callbacks 
earlystopping = callbacks.EarlyStopping(monitor ="loss", mode ="min", patience = 5, restore_best_weights = True)
callbacks =[earlystopping]

In [11]:
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save("chatbot_model.h5", hist)
print("model created")

Epoch 1/200
35/35 [==============================] - 1s 1ms/step - loss: 3.7220 - accuracy: 0.0183 
Epoch 2/200
35/35 [==============================] - 0s 2ms/step - loss: 3.5393 - accuracy: 0.0766
Epoch 3/200
35/35 [==============================] - 0s 2ms/step - loss: 3.3378 - accuracy: 0.2227
Epoch 4/200
35/35 [==============================] - 0s 1ms/step - loss: 3.1910 - accuracy: 0.1628
Epoch 5/200
35/35 [==============================] - 0s 1ms/step - loss: 2.9656 - accuracy: 0.2319
Epoch 6/200
35/35 [==============================] - 0s 1ms/step - loss: 2.8170 - accuracy: 0.2948
Epoch 7/200
35/35 [==============================] - 0s 1ms/step - loss: 2.4739 - accuracy: 0.4461
Epoch 8/200
35/35 [==============================] - 0s 1ms/step - loss: 2.0817 - accuracy: 0.5694
Epoch 9/200
35/35 [==============================] - 0s 1ms/step - loss: 2.1780 - accuracy: 0.3339
Epoch 10/200
35/35 [==============================] - 0s 2ms/step - loss: 1.9525 - accuracy: 0.5311
Epoch 11

In [12]:
model = load_model("chatbot_model.h5")
words = pickle.load(open("words.pkl", "rb"))
classes = pickle.load(open("classes.pkl", "rb"))

In [13]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words


# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    return np.array(bag)


In [14]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list


def getResponse(ints, intents_json):
    tag = ints[0]["intent"]
    list_of_intents = intents_json["intents"]
    for i in list_of_intents:
        if i["tag"] == tag:
            result = random.choice(i["responses"])
            break
    return result

In [15]:
# Test the chatbot with custom input
import time

msg = input()
if msg.startswith('my name is'):
    name = msg[11:]
    ints = predict_class(msg, model)
    res1 = getResponse(ints, intents)
    res =res1.replace("{n}",name)
elif msg.startswith('hi my name is'):
    name = msg[14:]
    ints = predict_class(msg, model)
    res1 = getResponse(ints, intents)
    res =res1.replace("{n}",name)
else:
    u = time.time()
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    print(time.time() - u)
print(res)


what measures can we take to prevent corona?
0.1123509407043457
1. Regularly and thoroughly clean your hands with an alcohol-based hand rub or wash them with soap and water.
2. Maintain at least 1 metre (3 feet) distance between yourself and anyone who is coughing or sneezing.
3. Avoid touching eyes, nose and mouth.
4. Make sure you, and the people around you, follow good respiratory hygiene. This means covering your mouth and nose with your bent elbow or tissue when you cough or sneeze. Then dispose of the used tissue immediately.
5. Stay home if you feel unwell. If you have a fever, cough and difficulty breathing, seek medical attention and call in advance. Follow the directions of your local health authority.
6. If possible, avoid traveling to places  – especially if you are an older person or have diabetes, heart or lung disease.
7. Keep up to date on the latest COVID-19 hotspots (cities or local areas where COVID-19 is spreading widely).
8. Self-isolate by staying at home if you b